In [4]:
%config Completer.use_jedi = False

In [5]:
import pickle
import numpy as np

In [16]:
import time

In [6]:
with open("label2num.pkl", "rb") as fp:   # Unpickling
    label2num = pickle.load(fp)
    
with open("num2label.pkl", "rb") as fp:   # Unpickling
    num2label = pickle.load(fp)
    
with open("train_feature_mock.pkl", "rb") as fp:   # Unpickling
    train_feature = pickle.load(fp)

with open("train_label_mock.pkl", "rb") as fp:   # Unpickling
    train_label = pickle.load(fp)   

In [9]:
points_np = np.concatenate(train_feature, axis=0)

In [10]:
points_np.shape

(1085888, 48)

In [12]:
len(label2num)

47

In [11]:
from sklearn.cluster import MiniBatchKMeans

In [52]:
number_of_clusters = 47

In [53]:
kmeans = MiniBatchKMeans(n_clusters=number_of_clusters,
                         random_state=2,
                         batch_size = 100000
                        )

In [183]:
kmeans.get_params()['n_clusters']

47

In [38]:
start = time.time()

In [39]:
kmeans = kmeans.fit(points_np)

In [40]:
end = time.time()
print(end-start)

0.7656826972961426


In [41]:
texton_label = kmeans.predict(points_np)

In [42]:
texton_label.shape

(1085888,)

In [45]:
train_texton_space_feature = []

start = 0
for i in range(len(train_feature)):
    end = train_feature[i].shape[0]+start
    train_texton_space_feature.append(texton_label[start:end])
    start = end
    

In [46]:
len(train_texton_space_feature)

188

In [75]:
train_feature_texto_histogram = []
for i in range(len(train_texton_space_feature)):
    unique, counts = np.unique(train_texton_space_feature[i], return_counts=True)
    histo_dict = dict(zip(unique, counts))
    
    histo_np = np.zeros(number_of_clusters).astype(np.float32)

    for j in range(number_of_clusters):
        if j in histo_dict:
            histo_np[j] = histo_dict[j]/counts.sum()
    train_feature_texto_histogram.append(np.clip(histo_np, 1e-7, np.inf))

In [64]:
train_feature_texto_histogram[0].shape

(47,)

In [76]:
train_feature_texto_histogram[0]

array([7.2714679e-02, 7.7042937e-02, 1.8698061e-02, 4.5013851e-03,
       8.5180059e-02, 1.5235457e-02, 4.5013851e-03, 3.4626038e-03,
       2.7354570e-02, 6.8213299e-02, 3.6357341e-03, 5.0207758e-03,
       1.0000000e-07, 3.2894737e-03, 8.8988923e-02, 1.3850415e-02,
       2.3199446e-02, 1.7313019e-04, 5.3670360e-03, 7.9639889e-03,
       1.9563712e-02, 3.4279779e-02, 4.6572022e-02, 1.0560942e-02,
       1.0000000e-07, 4.5013851e-03, 1.9044321e-03, 7.4445982e-03,
       2.1814404e-02, 1.0000000e-07, 9.6952906e-03, 1.0000000e-07,
       4.7610804e-02, 3.1163434e-03, 2.6142659e-02, 3.6011081e-02,
       1.0000000e-07, 1.3157895e-02, 1.5581718e-02, 2.5969530e-03,
       1.2638505e-02, 1.8005541e-02, 1.2984765e-02, 4.4148199e-02,
       1.0000000e-07, 1.5927978e-02, 6.7347646e-02], dtype=float32)

In [67]:
def cosine_similarity(list_1, list_2):
    cos_sim = np.dot(list_1, list_2) / (np.linalg.norm(list_1) * np.linalg.norm(list_2))
    return cos_sim

In [83]:
def chi2_distance(list_1, list_2):
    d = 0.5*( (list_1-list_2)**2 / (list_1+list_2) ).sum()
    return d

In [68]:
sim_matrix = np.zeros([len(train_feature_texto_histogram),len(train_feature_texto_histogram)])

for i in range(sim_matrix.shape[0]):
    for j in range(sim_matrix.shape[1]):
        sim_matrix[i,j] = cosine_similarity(train_feature_texto_histogram[i],train_feature_texto_histogram[j])

In [84]:
dist_matrix = np.zeros([len(train_feature_texto_histogram),len(train_feature_texto_histogram)])

for i in range(dist_matrix.shape[0]):
    for j in range(dist_matrix.shape[1]):
        dist_matrix[i,j] = chi2_distance(train_feature_texto_histogram[i],train_feature_texto_histogram[j])

In [69]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10, 10))
plt.imshow(sim_matrix, cmap="gray")
plt.show()

fig = plt.figure(figsize=(10, 10))
plt.imshow(dist_matrix, cmap="gray")
plt.show()

In [87]:
from sklearn.neighbors import KNeighborsClassifier

In [88]:
neigh = KNeighborsClassifier(n_neighbors=5, weights='distance', metric='precomputed')

In [89]:
neigh.fit(dist_matrix, train_label)

KNeighborsClassifier(metric='precomputed', weights='distance')

In [108]:
###query need to laod validation dataset, kmeans to find texton frequency, construct histogram, make distance matrix, predict
query = train_feature_texto_histogram


In [109]:


query_input = np.zeros([len(query),len(train_feature_texto_histogram)])

for i in range(query_input.shape[0]):
    for j in range(query_input.shape[1]):
        query_input[i,j] = chi2_distance(query[i],train_feature_texto_histogram[j])

In [94]:
query_input.shape

(2, 188)

In [110]:
label_predicted_prob = neigh.predict_proba(query_input)

In [111]:
label_predicted = label_predicted_prob.argmax(axis=1)

(188,)

In [133]:
def list_indices(list_1, element):
    indices = [i for i,x in enumerate(list_1) if x == element]
    return indices

In [136]:
u = np.unique(train_label)

In [138]:
u.shape

(47,)

In [170]:


#construct a matrix that store the value of true labels vs. predicted labels table
#axis0 is true label, axis1 is the predicted label
label_mat = np.zeros([len(label2num),len(label2num)])
label_prob_mat = np.zeros([len(label2num),len(label2num)])

for i in range(u.shape[0]):
    indices = list_indices(train_label, u[i])
    for index in indices:
        label_mat[u[i],label_predicted[index]] += 1
        label_prob_mat[u[i]] += label_predicted_prob[index]



In [ ]:
label_label_mat = label_mat/label_mat.sum(axis=1, keepdims=True)    
label_label_prob_mat = label_prob_mat/label_prob_mat.sum(axis=1, keepdims=True)

In [173]:
def confusion_mat_generating(labelXlabel_mat):
    label_confusion_mat = np.zeros([labelXlabel_mat.shape[0],2,2])

    """
    True Positive [0,0]
    False Negative [0,1]
    False Positive [1,0]
    True Negative [1,1]
    """

    for i in range(labelXlabel_mat.shape[0]):
        label_confusion_mat[i][0,0] = labelXlabel_mat[i,i]
        label_confusion_mat[i][0,1] = labelXlabel_mat[i,:].sum() - labelXlabel_mat[i,i] 
        label_confusion_mat[i][1,0] = labelXlabel_mat[:,i].sum() - labelXlabel_mat[i,i]
        label_confusion_mat[i][1,1] = labelXlabel_mat.sum() - labelXlabel_mat[i,:].sum() - labelXlabel_mat[:,i].sum() + labelXlabel_mat[i,i]
    
    return label_confusion_mat

In [180]:
def accuracy(confusion_mat):
    """
    return accuracy and precision
    
    confusion_mat with shape number_of_classes *2 *2
    
    return a matrix: accu_mat, with shape number_of_classes*2
    accu_mat[:,0] is precision
    accu_mat[:,1] is recall
    """
    accu_mat = np.zeros([confusion_mat.shape[0],2])
    for i in range(confusion_mat.shape[0]): 
        accu_mat[i,0] = confusion_mat[i,0,0]/(confusion_mat[i,0,0] + confusion_mat[i,1,0])
        accu_mat[i,1] = confusion_mat[i,0,0]/(confusion_mat[i,0,0] + confusion_mat[i,0,1])
    return accu_mat

In [178]:
confusion_mat_label = confusion_mat_generating(label_prob_mat)

In [179]:
confusion_mat_label[2]

array([[ 1.,  0.],
       [ 0., 46.]])

In [167]:
sum_of_mat = label_mat.sum(axis=1, keepdims=True)

In [168]:
sum_of_mat.shape

(47, 1)

In [169]:
(label_mat / sum_of_mat) [:10,:10]

array([[0.5, 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ]])

In [155]:
label_mat[0,0] = 1

In [156]:
label_mat.sum(axis=1) 

array([2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [ ]:
label_mat

In [157]:
label_mat[:10,:10]

array([[1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [ ]:
def label_accuracy_matrix_construct(true_label, predicted_label):
    

In [ ]:
def probability_accuracy_matrix_construct(true_label_prob, predicted_label_prob):

In [101]:
def accuracy(true_label, predicted_label):
    

(1085888, 48)